### Parte 1
    - coleta os IDs dos canais desejados
    - coleta as informações dos canais
    - coleta informações gerais de publicações
    - salva em um csv para cada canal com as informações específicas de cada canal / vídeo

In [ ]:
from googleapiclient.discovery import build
import csv
from datetime import datetime

# Configurações
DEVELOPER_KEY = "[CHAVE_REMOVIDA]"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# Lista de URLs dos canais para processar
channel_urls = [
    "www.youtube.com/@PadreFabiodeMeloOficial",
    "www.youtube.com/@BernardoKuster",
    "www.youtube.com/@PadrePatrickFernandes",
    "www.youtube.com/@padrepauloricardo",
    "https://www.youtube.com/@OArcanjoNoAr",
    "https://www.youtube.com/@leonardodallosto6884",
    "https://www.youtube.com/@padreleonardowagner",
    "https://www.youtube.com/@PadreJoseEduardo"
]

def youtube_service():
    return build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

def extract_channel_id(url):
    match = re.search(r"youtube\.com/[@](.+)", url)
    if match:
        return None, match.group(1)
    return None, None

def get_channel_id_from_custom_url(custom_url):
    youtube = youtube_service()
    request = youtube.channels().list(
        part="id,snippet",
        forUsername=custom_url
    )
    response = request.execute()

    if 'items' in response and response['items']:
        return response['items'][0]['id']
    else:
        return None

def search_channel_by_keyword(keyword):
    youtube = youtube_service()
    request = youtube.search().list(
        part="snippet",
        q=keyword,
        type="channel",
        maxResults=1
    )
    response = request.execute()

    if 'items' in response and response['items']:
        return response['items'][0]['id']['channelId']
    else:
        return None

def get_channel_info(channel_id):
    youtube = youtube_service()
    request = youtube.channels().list(
        part="snippet,statistics",
        id=channel_id
    )
    response = request.execute()

    channel_info = []
    for channel in response.get('items', []):
        data = {
            'Channel ID': channel['id'],
            'Nome do Canal': channel['snippet']['title'],
            'Descrição': channel['snippet']['description'],
            'Inscritos': channel['statistics'].get('subscriberCount', 'Privado'),
            'Visualizações': channel['statistics'].get('viewCount', 'Privado'),
            'Vídeos Publicados': channel['statistics'].get('videoCount', 'Privado')
        }
        channel_info.append(data)
    return channel_info

def get_videos(channel_id, start_date, end_date):
    youtube = youtube_service()
    request = youtube.search().list(
        part="snippet",
        channelId=channel_id,
        maxResults=50,
        type="video",
        publishedAfter=start_date,
        publishedBefore=end_date,
        order="date"
    )
    
    videos = []
    while request:
        response = request.execute()
        for item in response.get("items", []):
            video_id = item["id"]["videoId"]
            video_title = item["snippet"]["title"]
            video_published = item["snippet"]["publishedAt"]

            video_details = youtube.videos().list(
                part="statistics",
                id=video_id
            ).execute()

            stats = video_details["items"][0]["statistics"]
            view_count = stats.get("viewCount", 0)
            like_count = stats.get("likeCount", 0)
            comment_count = stats.get("commentCount", 0)

            video_info = {
                "URL": f"https://www.youtube.com/watch?v={video_id}",
                "Title": video_title,
                "Published Date": video_published,
                "Views": view_count,
                "Likes": like_count,
                "Comments": comment_count
            }
            videos.append(video_info)

        request = youtube.search().list_next(request, response) if "nextPageToken" in response else None

    return videos

def save_to_csv(data, filename):
    if data:
        keys = data[0].keys()
        with open(filename, 'w', newline='', encoding='utf-8') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(data)

def main():
    start_date = "2024-01-01T00:00:00Z"
    end_date = "2024-05-05T00:00:00Z"

    # Lista para o resumo geral
    general_summary = []

    # Coletar IDs dos canais e suas informações
    channel_info_list = []
    channel_ids = []

    for url in channel_urls:
        _, possible_name = extract_channel_id(url)
        if possible_name:
            channel_id = get_channel_id_from_custom_url(possible_name)
            if not channel_id:
                channel_id = search_channel_by_keyword(possible_name)
            if channel_id:
                channel_ids.append(channel_id)
                channel_info = get_channel_info(channel_id)
                channel_info_list.extend(channel_info)

    # Salvar informações dos canais em CSV
    save_to_csv(channel_info_list, 'channel_info.csv')

    # Coletar informações detalhadas dos vídeos para cada canal
    for channel_info in channel_info_list:
        channel_id = channel_info['Channel ID']
        channel_name = channel_info['Nome do Canal']
        videos = get_videos(channel_id, start_date, end_date)

        # Resumo geral para cada canal
        total_videos = len(videos)
        total_likes = sum(int(video["Likes"]) for video in videos)
        general_summary.append({
            "Nome do Canal": channel_name,
            "Total de Vídeos": total_videos,
            "Total de Curtidas": total_likes
        })

        # Salvar informações detalhadas dos vídeos para cada canal
        filename = f"{channel_name.replace(' ', '_')}_videos.csv"
        save_to_csv(videos, filename)

    # Salvar resumo geral em CSV
    save_to_csv(general_summary, 'general_summary.csv')

if __name__ == "__main__":
    main()


#### 2 - Extrair audios dos vídeos e salvar localmente

In [ ]:
import csv
import subprocess
import os

# Configurações
csv_path = 'CSVs/Padre_José_Eduardo_videos.csv'
output_dir = 'Audios/Padre_José_Eduardo'   
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def download_and_convert(url, output_filename):
    try:
        # Comando para baixar o áudio usando yt-dlp e enviar para ffmpeg via pipe
        command = f'yt-dlp -f bestaudio "{url}" -o - | ffmpeg -i pipe: -b:a 192k -vn "{output_filename}"'
        subprocess.run(command, shell=True, check=True)
        print(f'Áudio baixado e convertido para {output_filename}')
    except subprocess.CalledProcessError as e:
        print(f"Erro ao baixar ou converter vídeo: {url}. Erro: {str(e)}")

def process_videos(csv_path, output_dir):
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            url = row['URL']  # Assumindo que a coluna com URLs se chama 'URL'
            video_id = url.split('=')[-1]  # Extrai o ID do vídeo para usar como nome de arquivo
            output_filename = os.path.join(output_dir, f'{video_id}.mp3')
            download_and_convert(url, output_filename)

if __name__ == '__main__':
    process_videos(csv_path, output_dir)


### 3 - transcrição do vídeo - funciona com cpu

In [ ]:
!pip install git+https://github.com/openai/whisper.git


In [ ]:
import os
import csv
import whisper
def transcribe_audio(model, audio_file_path):
    result = model.transcribe(audio_file_path, language="Portuguese")
    return result['text']

def process_directory(audio_directory, output_csv_path, model_size='medium'):
    # Carregar o modelo Whisper
    model = whisper.load_model(model_size)
    
    # Criar ou abrir o arquivo CSV para escrita
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Filename', 'Transcription'])  # Cabeçalhos das colunas

        # Iterar sobre todos os arquivos no diretório especificado
        for filename in os.listdir(audio_directory):
            if filename.lower().endswith((".mp3", ".wav")):  # Suporta múltiplos formatos de áudio
                file_path = os.path.join(audio_directory, filename)
                print(f"Transcrevendo {filename}...")
                # Transcrever o áudio
                transcription = transcribe_audio(model, file_path)
                # Escrever o resultado no CSV
                csv_writer.writerow([filename, transcription])
                print(f"Transcrição completa para {filename}")

if __name__ == '__main__':
    directory = 'Audios/pe.arcanjo'
    output_csv = 'm_transc_pe_arcanjo.csv'
    process_directory(directory, output_csv, model_size='medium')


### funciona com GPU 

In [2]:
import os
import csv
import whisper
import torch

def transcribe_audio(model, audio_file_path):
    try:
        # Carregar o áudio com suporte a GPU se disponível
        audio = whisper.load_audio(audio_file_path)
        audio = whisper.pad_or_trim(audio)
        # Mover o tensor de áudio para GPU se disponível
        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        # Executar a transcrição com o modelo
        options = whisper.DecodingOptions(fp16=True)
        result = model.decode(mel, options)
        return result.text
    except Exception as e:
        print(f"Erro ao processar {audio_file_path}: {e}")
        return None

def process_directory(audio_directory, output_csv_path, model_size='medium'):
    # Verificar se CUDA está disponível e carregar o modelo apropriadamente
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando o dispositivo: {device}")
    model = whisper.load_model(model_size, device=device)

    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['Filename', 'Transcription'])

        for filename in os.listdir(audio_directory):
            if filename.lower().endswith((".mp3", ".wav")):
                file_path = os.path.join(audio_directory, filename)
                print(f"Transcrevendo {filename}...")
                transcription = transcribe_audio(model, file_path)
                if transcription is not None:
                    csv_writer.writerow([filename, transcription])
                    print(f"Transcrição completa para {filename}")
                else:
                    print(f"Falha ao transcrever {filename}")

    # Descarregar o modelo da memória e liberar memória da GPU
    del model
    torch.cuda.empty_cache()
    print("Memória da GPU liberada.")

# Executar a função diretamente
directory = 'Audios/Pe.Leonardo'
output_csv = 'm_transc_Pe.Leonardo.csv'
process_directory(directory, output_csv, model_size='medium')

Usando o dispositivo: cuda
Transcrevendo FMtPvZ6jg-o.mp3...
Transcrição completa para FMtPvZ6jg-o.mp3
Transcrevendo e_ja3_eAZ80.mp3...
Transcrição completa para e_ja3_eAZ80.mp3
Transcrevendo 7g6RY2qiRkc.mp3...
Transcrição completa para 7g6RY2qiRkc.mp3
Transcrevendo ePaS7ay9qKo.mp3...
Transcrição completa para ePaS7ay9qKo.mp3
Transcrevendo 9JNpgpbr1B4.mp3...
Transcrição completa para 9JNpgpbr1B4.mp3
Transcrevendo FrJnzKtdotU.mp3...
Transcrição completa para FrJnzKtdotU.mp3
Transcrevendo qGhu26AlqDk.mp3...
Transcrição completa para qGhu26AlqDk.mp3
Transcrevendo s9hHO0xiXg0.mp3...
Transcrição completa para s9hHO0xiXg0.mp3
Transcrevendo hGsRTme1cb0.mp3...
Transcrição completa para hGsRTme1cb0.mp3
Transcrevendo FKiXRW8Bu8U.mp3...
Transcrição completa para FKiXRW8Bu8U.mp3
Transcrevendo wGYdOHb-QnA.mp3...
Transcrição completa para wGYdOHb-QnA.mp3
Transcrevendo 2vPqZboE0sc.mp3...
Transcrição completa para 2vPqZboE0sc.mp3
Transcrevendo QJTJdpXXswk.mp3...
Transcrição completa para QJTJdpXXswk.mp